## Задача

Восстановить значения функции f(x) в заданных точках x*, x**, x*** используя интерполяционный многочлен Ньютона
<br> Функция $f(x) = \alpha_j * e^x + (1-\alpha_j) * sin(x)$ где $\alpha_j = 0.7$ принимает вид: 
$$ f(x) = 0.7 * e^x + 0.3 * sin(x)$$
задана на отрезке [0.7; 1.7]
<br><br>
Необходимо найти значения функции в точках:
$$x_i = \alpha_j + i/10, i = 1, 2, ..., 10$$
и на основании полученных значений построить интерполяционный многочлен Ньютона и с его помощью найти значения функции f(x) в точках: $$x^* = x_0 + 2/30$$ $$x^{**} = x_n / 2 + 1/20$$ $$x^{***} = x_n - 1/30$$

In [58]:
%pip install tabulate
%pip install sympy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [59]:
import numpy as np
import math
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp

In [60]:
j = 12
n = 10
alpha_j = 0.1 + 0.05*j
h = 1/n

In [61]:
def f(x):
    return alpha_j * math.exp(x) + (1 - alpha_j) * math.sin(x)

Шаг 1. Построим исходную таблицу

In [62]:

x_vals = np.array([alpha_j + i * h for i in range(n+1)])
f_vals = np.array([f(x_) for x_ in x_vals])
x_star = np.array([x_vals[0] + 2*h/3, x_vals[len(x_vals) // 2] + h/2, x_vals[-1] - h/3])
f_star = np.array([f(x_) for x_ in x_star])

print(tabulate(zip(x_vals, f_vals), headers=['x', 'f(x)']))
print('\nСпециальные точки')
print(tabulate(zip(x_star, f_star), headers=['x*', 'f(x*)']))

  x     f(x)
---  -------
0.7  1.60289
0.8  1.77309
0.9  1.95672
1    2.15524
1.1  2.37028
1.2  2.60369
1.3  2.85758
1.4  3.13427
1.5  3.43643
1.6  3.76699
1.7  4.12926

Специальные точки
      x*    f(x*)
--------  -------
0.766667  1.71493
1.25      2.72794
1.66667   4.00477


Строим интерполяционный многочлен

In [67]:
# Таблица значений функции
table = pd.DataFrame({"x_i": x_vals, "f(x_i)": f_vals})
table_transposed = table.T

def compute_newton_coefficients(x_vals, y_vals):
    """
    Возвращает список коэффициентов интерполяционного многочлена Ньютона
    с использованием рекурсивного определения разделённых разностей.
    """
    n = len(x_vals)
    # Создаём таблицу размером n x n
    dd_table = [y_vals.copy()]  # f[x_i]

    for level in range(1, n):
        prev_column = dd_table[-1]
        curr_column = []
        for i in range(len(prev_column)-1):
            numerator = prev_column[i + 1] - prev_column[i]
            denominator = x_vals[i + level] - x_vals[i]
            curr_column.append(numerator / denominator)
        dd_table.append(curr_column)
    # Коэффициенты Ньютона — это верхние элементы каждого столбца
    return dd_table, [dd_table[i][0] for i in range(n)]
    

def extend_divided_difference(dd_table, x_vals, x_star, f_star):
    """
    Расширяет таблицу разделённых разностей на одну точку x_star, f_star
    и возвращает f[x0, ..., xn, x*] (верхний элемент новой диагонали).
    """
#    n = len(x_vals)
    column = [f_star.copy()]
    for k in range(len(x_vals)):
        numerator = column[-1] - dd_table[k][-1]
        denominator = x_star - x_vals[k]
        column.append(numerator / denominator)
    return column[-1]




dd_table, newton_coeffs = compute_newton_coefficients(x_vals, f_vals)
def newton_interpolation(x_vals, y_vals, x, coef):
    """
    Вычисляет значение интерполяционного многочлена Ньютона в точке x
    с использованием рекурсивной формулы:
    P_{n+1}(x) = P_n(x) + alpha_{n+1} * omega_{n+1}(x)
    """
    result = coef[0]
    omega = 1.0
    for i in range(1, len(coef)):
        omega *= (x - x_vals[i - 1])
        result += coef[i] * omega
    return result

def omega(x_vals, x_point):
    res = 1
    for x in x_vals:
        res *= (x_point - x)
    return res
omegas = [omega(x_vals, x_point) for x_point in x_star]

P_x_star = [newton_interpolation(x_vals, f_vals, x_st, newton_coeffs) for x_st in x_star]

omega_frame = pd.DataFrame(omegas, index=[f'omega_{i}' for i in range(len(omegas))])
# Результаты интерполяции
data = {
    "Точка": ["x*", "x**", "x***"],
    "Значение x": x_star,
    "f(x)": f_star,
    "P(x) (полином)": P_x_star
}

n = len(newton_coeffs)
dd_frame = pd.DataFrame(dd_table).T
dd_frame.columns = [f"f[x0..x{i}]"
                      for i, n in zip(range(len(dd_table)), reversed(range(len(dd_table))))]

dd_frame.insert(0, "x_i", x_vals)
coeff_frame = pd.DataFrame(newton_coeffs, index=[f"a_{i}" for i in range(n)]).T
df = pd.DataFrame(data)

# Истинная погрешность
r_x_stars = np.abs(f_star - P_x_star)

error_bound_stars = []

for i in range(len(x_star)):
    error_bound = abs(extend_divided_difference(dd_table, x_vals, x_star[i], f_star[i]) * omegas[i]) 
    error_bound_stars.append(error_bound)

# Проверка выполнения неравенства
is_error_bound_stars_valid = [
    abs(r_x_stars[i]) <= error_bound_stars[i] for i in range(3)
]

# Таблица ошибок
error_table = pd.DataFrame({
    "Точка": ["x*", "x**", "x***"],
    "Значение x": x_star,
    "r истинная": r_x_stars,
    "оценка погрешности": error_bound_stars,
    "Неравенство выполняется?": is_error_bound_stars_valid
})

# Вывод таблиц
display(table_transposed)
display(df)
display(error_table)
display(dd_frame)
display(coeff_frame)
display(omega_frame)




,0,1,2,3,4,5,6,7,8,9,10
x_i,0.700000,0.800000,0.90000,1.000000,1.100000,1.200000,1.300000,1.400000,1.500000,1.600000,1.700000
f(x_i),1.602892,1.773085,1.95672,2.155239,2.370278,2.603694,2.857575,3.134275,3.436431,3.766995,4.129263


,Точка,Значение x,f(x),P(x) (полином)
0,x*,0.766667,1.714927,1.714927
1,x**,1.250000,2.727935,2.727935
2,x***,1.666667,4.004765,4.004765


,Точка,Значение x,r истинная,оценка погрешности,Неравенство выполняется?
0,x*,0.766667,1.021405e-13,2.652478,True
1,x**,1.250000,3.108624e-15,3.836909,True
2,x***,1.666667,2.433609e-13,5.375227,True


,x_i,f[x0..x0],f[x0..x1],f[x0..x2],f[x0..x3],f[x0..x4],f[x0..x5],f[x0..x6],f[x0..x7],f[x0..x8],f[x0..x9],f[x0..x10]
0,0.7,1.602892,1.701933,0.672075,0.240342,0.081633,0.016566,0.002300,0.000369,0.000059,0.000006,5.661216e-07
1,0.8,1.773085,1.836348,0.744178,0.272996,0.089916,0.017946,0.002558,0.000416,0.000065,0.000007,NaN
2,0.9,1.956720,1.985183,0.826076,0.308962,0.098889,0.019480,0.002849,0.000467,0.000071,NaN,NaN
3,1.0,2.155239,2.150398,0.918765,0.348518,0.108629,0.021189,0.003176,0.000524,NaN,NaN,NaN
4,1.1,2.370278,2.334151,1.023320,0.391970,0.119224,0.023095,0.003543,NaN,NaN,NaN,NaN
5,1.2,2.603694,2.538816,1.140911,0.439659,0.130772,0.025221,NaN,NaN,NaN,NaN,NaN
6,1.3,2.857575,2.766998,1.272809,0.491968,0.143382,NaN,NaN,NaN,NaN,NaN,NaN
7,1.4,3.134275,3.021559,1.420399,0.549321,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.5,3.436431,3.305639,1.585195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.6,3.766995,3.622678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10
0,1.602892,1.701933,0.672075,0.240342,0.081633,0.016566,0.0023,0.000369,0.000059,0.000006,5.661216e-07


,0
omega_0,1.923942e-06
omega_1,-4.796521e-08
omega_2,-4.100672e-06


In [ ]:
dd_table[0]

array([1.6028922 , 1.77308548, 1.95672025, 2.15523858, 2.37027842,
       2.60369357, 2.85757512, 3.1342749 , 3.43643085, 3.76699478,
       4.12926262])